In [9]:
# Dataframe from NSE Option Chains

# STATUS: Complete

import requests
import lxml.html as lh
import datetime as dt
import pandas as pd
import numpy as np

symbol = 'NIFTY'  # Sample symbol

url_base = "https://nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol="
url_end = '&date='
url = url_base + symbol

page = requests.get(url)
doc = lh.fromstring(page.content)

#...    Get the Expiries ...#
#____________________________

the_path= "//form"   # Get the form content. There are two forms
expiries = doc.xpath(the_path)[0].text_content()
strip_chars = [ord('\n'), ord('\xa0'), ord('\t'), ord('\r')] # characters to be stripped
char_table = {s: ' ' for s in strip_chars} # table for translate to locate the chars

sym_exp = expiries.translate(char_table).split() # split converts the translated dict into list

# Get the expiries only, from the table
expiry = [sym_exp[k+i] 
 for k, v in enumerate(sym_exp) 
 if v in 'Select' 
 for i in range(len(sym_exp) - k)][1:]

# Convert expiry to datetime
expiry_dt = [dt.datetime.strptime(date, "%d%b%Y").date() for date in expiry]

#...   Get the Options Data .... #
#_________________________________

tbl_path = "//*[@id='octable']"  # xpath for the table

# The headers
opt_head = ['cOI', 'cChnginOI', 'cVolume', 'cIV', 'cLTP', 
            'cNetChng', 'cBidQty', 'cBidPrice', 'cAskPrice', 'cAskQty', 'Strike', 
            'pBidQty', 'pBidPrice', 'pAskPrice', 'pAskQty', 'pNetChng', 'pLTP', 
            'pIV', 'pVolume', 'pChnginOI', 'pOI']

opt_data = [tr.text_content() for table in doc.xpath(tbl_path) for tr in table[1:]]

df1 = pd.DataFrame([d.split() for d in opt_data], columns=opt_head)

df2 = df1.replace(',', '', regex=True) # Remove comma from numbers
df2 = df2.apply(pd.to_numeric, errors='coerce') # Convert to numeric
df2.insert(0, 'Expiry', expiry_dt[1]) # Insert the Expiry column
df2.insert(0, 'nseSymbol', symbol) # Insert the Symbol

# Rearrange the columns
cols_beginning = ['nseSymbol', 'Expiry', 'Strike']
df2 = df2[cols_beginning + [c for c in df2 if c not in cols_beginning]]

In [10]:
df2

,nseSymbol,Expiry,Strike,cOI,cChnginOI,cVolume,cIV,cLTP,cNetChng,cBidQty,...,pBidQty,pBidPrice,pAskPrice,pAskQty,pNetChng,pLTP,pIV,pVolume,pChnginOI,pOI
0,NIFTY,2018-12-27,8550.0,75.0,NaN,NaN,NaN,2205.95,NaN,225.0,...,300.0,0.60,1.50,75.0,-0.10,0.55,52.28,29.0,150.0,825.0
1,NIFTY,2018-12-27,8600.0,225.0,NaN,2.0,NaN,2005.00,28.60,600.0,...,75.0,0.60,1.40,1500.0,-0.70,0.60,51.43,34.0,-1500.0,215850.0
2,NIFTY,2018-12-27,8650.0,NaN,NaN,NaN,NaN,NaN,NaN,1200.0,...,375.0,1.80,2.00,75.0,1.05,1.80,56.39,20.0,-1350.0,11025.0
3,NIFTY,2018-12-27,8700.0,900.0,NaN,NaN,NaN,1344.40,NaN,600.0,...,1800.0,1.20,1.30,2625.0,-0.25,1.30,52.96,1479.0,-1050.0,675150.0
4,NIFTY,2018-12-27,8750.0,2850.0,NaN,2.0,72.58,1883.75,11.40,150.0,...,150.0,0.60,1.00,375.0,NaN,0.60,47.57,5.0,-225.0,3525.0
5,NIFTY,2018-12-27,8800.0,1200.0,NaN,NaN,NaN,1242.60,NaN,600.0,...,1500.0,1.15,1.40,300.0,-0.35,1.20,49.78,73.0,450.0,540000.0
6,NIFTY,2018-12-27,8850.0,NaN,NaN,NaN,NaN,NaN,NaN,150.0,...,75.0,1.05,2.50,75.0,NaN,2.50,NaN,NaN,NaN,300.0
7,NIFTY,2018-12-27,8900.0,150.0,NaN,NaN,NaN,1165.45,NaN,600.0,...,1050.0,1.20,1.30,75.0,-0.55,1.05,46.40,216.0,-525.0,266250.0
8,NIFTY,2018-12-27,8950.0,NaN,NaN,NaN,NaN,NaN,NaN,150.0,...,75.0,0.70,2.45,3000.0,NaN,1.25,NaN,NaN,NaN,300.0
9,NIFTY,2018-12-27,9000.0,434700.0,-22800.0,578.0,47.45,1621.85,-46.60,75.0,...,675.0,1.20,1.35,2400.0,-0.20,1.35,45.01,3846.0,-63000.0,2032875.0
